# <center> Coursera Capstone Project </center>

## <center> Where Should I move? </center>

![alt text][logo]

[logo]: https://i2.wp.com/movingtips.wpengine.com/wp-content/uploads/2019/02/moving-boxes-crosscountry.jpg "Picture from Moving.com"


### Introduction/Business Problem
The stakeholders in this case are my girlfriend and I. We have both recently graduated from undergrad and have been debating where to live. We want to live somewhere near family, but also somewhere that is similar to our current situation. We live in Downtown Frederick currently and we really enjoy it. We have to move for her graduate program soon though and want to make the best decision on where to move.

With the knowledge that I've gained throughout these courses, I thought that it would be interesting to apply data science skills and the Foursquare location data to make a more informed decision. I have talked with her and she has helped me create a list of possible places that we could live. I want to perform a cluster analysis on these cities to see which of them tend to be like Downtown Frederick.

With the knowledge of which places are more similar to Frederick, we can make a more informed decision that will lead to greater satisfaction with our move. If there are others in a same situation, this mehtod may help you find similar living spaces.


### Data
This project requires that we use the Foursquare location data, which is perfect for the problem at hand. The Foursquare explore endpoint is of particular interest for this project. The explore endpoint allows for the user to specify where to search (latitude and longitude), how far around the given point to search (radius), how many venues to retreive (limit), and how to choose those top values (relevance, popularity, or distance). This call to the API will return the venues and information that meets all of those parameters. The venue information will include the location, category, and tips, among other things. Category is the main feature that we are interested in.

I hope to make a search for each of these cities on my list to obtain the top 100 most popular venues within a 5 kilometer radius of the center. I will then find the proportion of the categories for each city and use that as the feature set for the clustering algorithm. I will use K-means clustering to determine similar groups of cities based on their venues and then examine to see which group contains Frederick. Looking into that group will give me more information about what Frederick looks like and what similar cities tend to look like as well based on the venues that they have around.